<a href="https://colab.research.google.com/github/cs432-websci-master/public/blob/main/432_PCI_Ch08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-Nearest Neighbors

Ch 8 from *Programming Collective Intelligence*, based on code from
* https://github.com/arthur-e/Programming-Collective-Intelligence/tree/master/chapter8
* https://go.oreilly.com/old-dominion-university/library/view/programming-collective-intelligence/9780596529321/

**Goal:** Predict the price of a bottle of wine.

# Build the Dataset

Generate a dataset of wine ratings, age, and prices (pgs. 168-169)

In [ ]:
from random import random,randint
import math

In [ ]:
def wineprice(rating,age):
  peak_age=rating-50
  
  # Calculate price based on rating
  price=rating/2
  if age>peak_age:
    # Past its peak, goes bad in 10 years
    price=price*(5-(age-peak_age)/2)
  else:
    # Increases to 5x original value as it
    # approaches its peak
    price=price*(5*((age+1)/peak_age))
  if price<0: price=0
  return price

In [ ]:
def wineset1():
  rows=[]
  for i in range(300):
    # Create a random age and rating
    rating=random()*50+50
    age=random()*50

    # Get reference price
    price=wineprice(rating,age)
    
    # Add some noise
    #price*=(random()*0.2+0.9)
    price*=(random()*0.4+0.8)  # changed to match p.168 code

    # Add to the dataset
    rows.append({'input':(rating,age),
                 'result':price})
  return rows

In [ ]:
data = wineset1()
data[0]

{'input': (64.30597436892485, 49.5844073498069), 'result': 0.0}

# Distance Functions

In [ ]:
def euclidean(v1,v2):
  d=0.0
  for i in range(len(v1)):
    d+=(v1[i]-v2[i])**2
  return math.sqrt(d)

In [ ]:
def getdistances(data,vec1):
  distancelist=[]
  
  # Loop over every item in the dataset
  for i in range(len(data)):
    vec2=data[i]['input']
    
    # Add the distance and the index
    distancelist.append((euclidean(vec1,vec2),i))
  
  # Sort by distance
  distancelist.sort()
  return distancelist

# Weight Functions

In [ ]:
def inverseweight(dist,num=1.0,const=0.1):
  return num/(dist+const)

In [ ]:
def subtractweight(dist,const=1.0):
  if dist>const: 
    return 0
  else: 
    return const-dist

In [ ]:
def gaussian(dist,sigma=5.0):
  return math.e**(-dist**2/(2*sigma**2))

# kNN Estimate

In [ ]:
def knnestimate(data,vec1,k=5):
  # Get sorted distances
  dlist=getdistances(data,vec1)
  avg=0.0
  
  # Take the average of the top k results
  for i in range(k):
    idx=dlist[i][1]
    avg+=data[idx]['result']
  avg=avg/k
  return avg

In [ ]:
def weightedknn(data,vec1,k=5,weightf=gaussian):
  # Get distances
  dlist=getdistances(data,vec1)
  avg=0.0
  totalweight=0.0
  
  # Get weighted average
  for i in range(k):
    dist=dlist[i][0]
    idx=dlist[i][1]
    weight=weightf(dist)
    avg+=weight*data[idx]['result']
    totalweight+=weight
  if totalweight==0: return 0
  avg=avg/totalweight
  return avg

## Examples - Weighted vs. Non-Weighted 

In [ ]:
data = wineset1()

In [ ]:
wineprice(99.0, 5.0)

In [ ]:
knnestimate(data,(99.0,5.0))

In [ ]:
weightedknn(data,(99.0,5.0))

In [ ]:
weightedknn(data,(99.0,5.0), k=10)

In [ ]:
weightedknn(data,(99.0,5.0), k=20)

# Cross-Validation

In [ ]:
def dividedata(data,test=0.05):
  trainset=[]
  testset=[]
  for row in data:
    if random()<test:
      testset.append(row)
    else:
      trainset.append(row)
  return trainset,testset

In [ ]:
def testalgorithm(algf,trainset,testset):
  error=0.0
  for row in testset:
    guess=algf(trainset,row['input'])
    error+=(row['result']-guess)**2
    #print row['result'],guess
  #print error/len(testset)
  return error/len(testset)

In [ ]:
def crossvalidate(algf,data,trials=100,test=0.1):
  error=0.0
  for i in range(trials):
    trainset,testset=dividedata(data,test)
    error+=testalgorithm(algf,trainset,testset)
  return error/trials

## Examples - Cross-Validation

In [ ]:
def knn3(d,v): return knnestimate(d,v,k=3)

def knn1(d,v): return knnestimate(d,v,k=1) 

def wknn3(d,v): return weightedknn(d,v,k=3)

def wknn1(d,v): return weightedknn(d,v,k=1)

def wknn5inverse(d,v): return weightedknn(d,v,weightf=inverseweight)

In [ ]:
crossvalidate(knnestimate,data)

In [ ]:
crossvalidate(knn3,data)

In [ ]:
crossvalidate(knn1,data) 

In [ ]:
crossvalidate(weightedknn,data)

In [ ]:
crossvalidate(wknn3,data) 

In [ ]:
crossvalidate(wknn1,data)

In [ ]:
crossvalidate(wknn5inverse,data) 

# Rescaling

In [ ]:
def wineset2():
  rows=[]
  for i in range(300):
    rating=random()*50+50
    age=random()*50
    aisle=float(randint(1,20))
    bottlesize=[375.0,750.0,1500.0][randint(0,2)]
    price=wineprice(rating,age)
    price*=(bottlesize/750)
    price*=(random()*0.2+0.9)
    rows.append({'input':(rating,age,aisle,bottlesize),
                 'result':price})
  return rows

In [ ]:
def rescale(data,scale):
  scaleddata=[]
  for row in data:
    scaled=[scale[i]*row['input'][i] for i in range(len(scale))]
    scaleddata.append({'input':scaled,'result':row['result']})
  return scaleddata

## Examples - Rescaling

In [ ]:
data2 = wineset2() 

In [ ]:
data2[0:4]

In [ ]:
crossvalidate(knn3,data) 

In [ ]:
crossvalidate(knn3,data2) 

In [ ]:
crossvalidate(weightedknn,data)

In [ ]:
crossvalidate(weightedknn,data2)

In [ ]:
sdata=rescale(data2,[10,10,0,0.5])

In [ ]:
crossvalidate(knn3,sdata) 

In [ ]:
crossvalidate(weightedknn,sdata) 

# Optimization

In [ ]:
def createcostfunction(algf,data):
  def costf(scale):
    sdata=rescale(data,scale)
    return crossvalidate(algf,sdata,trials=20)
  return costf

In [ ]:
weightdomain=[(0,20)]*4

## Optimization Code from Ch 5

https://github.com/arthur-e/Programming-Collective-Intelligence/blob/master/chapter5

Notes: 
* Because we've already imported `random` and `randint` (top cell), I've removed the `random.` in front of all of the calls to `random` and `randint` in these two functions.
* In geneticoptimize(), I added parenthesis in the `print` statement and fixed a typo (`mutprob` -> `mutprod`)

In [ ]:
def annealingoptimize(domain,costf,T=10000.0,cool=0.95,step=1):
  # Initialize the values randomly
  vec=[float(randint(domain[i][0],domain[i][1])) 
       for i in range(len(domain))]
  
  while T>0.1:
    # Choose one of the indices
    i=randint(0,len(domain)-1)

    # Choose a direction to change it
    dir=randint(-step,step)

    # Create a new list with one of the values changed
    vecb=vec[:]
    vecb[i]+=dir
    if vecb[i]<domain[i][0]: vecb[i]=domain[i][0]
    elif vecb[i]>domain[i][1]: vecb[i]=domain[i][1]

    # Calculate the current cost and the new cost
    ea=costf(vec)
    eb=costf(vecb)
    p=pow(math.e,(-eb-ea)/T)

    # Is it better, or does it make the probability
    # cutoff?
    if (eb<ea or random()<p):
      vec=vecb      

    # Decrease the temperature
    T=T*cool
  return vec

In [ ]:
def geneticoptimize(domain,costf,popsize=50,step=1,
                    mutprod=0.2,elite=0.2,maxiter=100):
  # Mutation Operation
  def mutate(vec):
    i=randint(0,len(domain)-1)
    if random()<0.5 and vec[i]>domain[i][0]:
      return vec[0:i]+[vec[i]-step]+vec[i+1:] 
    elif vec[i]<domain[i][1]:
      return vec[0:i]+[vec[i]+step]+vec[i+1:]
  
  # Crossover Operation
  def crossover(r1,r2):
    i=randint(1,len(domain)-2)
    return r1[0:i]+r2[i:]

  # Build the initial population
  pop=[]
  for i in range(popsize):
    vec=[randint(domain[i][0],domain[i][1]) 
         for i in range(len(domain))]
    pop.append(vec)
  
  # How many winners from each generation?
  topelite=int(elite*popsize)
  
  # Main loop 
  for i in range(maxiter):
    scores=[(costf(v),v) for v in pop]
    scores.sort()
    ranked=[v for (s,v) in scores]
    
    # Start with the pure winners
    pop=ranked[0:topelite]
    
    # Add mutated and bred forms of the winners
    while len(pop)<popsize:
      if random()<mutprod:

        # Mutation
        c=randint(0,topelite)
        pop.append(mutate(ranked[c]))
      else:
      
        # Crossover
        c1=randint(0,topelite)
        c2=randint(0,topelite)
        pop.append(crossover(ranked[c1],ranked[c2]))
    
    # Print current best score
    print (scores[0][0])
    
  return scores[0][1]

## Examples - Optimization

In [ ]:
costf=createcostfunction(knnestimate,data2) 

In [ ]:
annealingoptimize(weightdomain,costf,step=2)

In [ ]:
annealingoptimize(weightdomain,costf,step=2) 

In [ ]:
annealingoptimize(weightdomain,costf,step=2) 

In [ ]:
geneticoptimize(weightdomain,costf,popsize=5)